In [3]:
_from_tweet = 1200000
_packet_size = 10

In [32]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
!pip install pytypo
!pip install pyspellchecker
!pip install bs4

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/learnsity/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/learnsity/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


  Using cached https://files.pythonhosted.org/packages/3b/62/e9da86d71e3ccc500b979f0afb88c1f3ae151766004a0de92775b686a311/pyspellchecker-0.5.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f9/d9/183705a87492249b212d88eef740995f55076195bcf45ed59306c146e42d/beautifulsoup4-4.8.1-py2-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5d/42/d821581cf568e9b7dfc5b415aa61952b0f5e3dede4f3cbd650e3a1082992/soupsieve-1.9.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/03/8e/2424c0e65c4a066e28f539364deee49b6451f8fcd4f718fefa50cc3dcf48/backports.functools_lru_cache-1.5-py2.py3-none-any.whl


In [33]:
import pandas as pd  
import numpy as np
_cols = ['sentiment','id','date','query_string','user','text']
_df = pd.read_csv('data.csv',header=None, names=_cols, low_memory=False)
_df = _df.drop(columns=['date', 'query_string','user'])
_df.head()

,sentiment,id,text
0,0,1467810369,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,is upset that he can't update his Facebook by ...
2,0,1467810917,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,my whole body feels itchy and like its on fire
4,0,1467811193,"@nationwideclass no, it's not behaving at all...."


In [35]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import WordNetLemmatizer 
from bs4 import BeautifulSoup
from spellchecker import SpellChecker
import string
import re
import pytypo
import json

# Tokenizers
_tweetok = TweetTokenizer()
_wptok = WordPunctTokenizer()

# Contractions Dictionary
_contraction_dict = json.load(open('contraction_dict.json', 'r'))

# Stop Words
_stop_words = set(stopwords.words('english'))
_exclude = ["not", "no", "should"]
_stop_words = [w for w in _stop_words if not w in _exclude]

# Regular Expressions
_r1 = r'@[A-Za-z0-9_]+' # Twitter username
_r2 = r'http\S+|www\S+|[A-Za-z0-9\/.:]*\.com(\.[A-Za-z]+)*' # URLs
_r3 = r'#[A-Za-z0-9]+' # HashTags
_final_regex = r'|'.join((_r1, _r2, _r3)) # Final RegEx

# Spell Checker
_spell = SpellChecker()
_spell.word_frequency.load_words(['microsoft', 'apple', 'google','twitter', 'facebook', 'amazon', 'texting', 'nah'])

# Lemmatization
_lemmatizer = WordNetLemmatizer()

In [36]:
def preprocess_tweet(valTweet):
    
    # HTML decoding
    valTweet = BeautifulSoup(valTweet, 'lxml').get_text()
    
    # Weird characters
    valTweet = ''.join(filter(lambda x: x in string.printable, valTweet))
    
    # URLs, mentions and hashtags
    valTweet = re.sub(_final_regex, '', valTweet)
    
    # To lowercase
    valTweet = valTweet.lower()
    
    # Expand contractions
    vTokens = _tweetok.tokenize(valTweet)
    valTweet = ' '.join([_contraction_dict[w] if w in _contraction_dict else w for w in vTokens]).strip()
    
    # Only alpha characters
    valTweet = re.sub(r'[^a-zA-Z]', ' ', valTweet)
    
    # Typos correction
    vTokens = _wptok.tokenize(valTweet)
    vTokens = [pytypo.correct(w) for w in vTokens]
    
    # Lonely letters removal
    vTokens = [w for w in vTokens if len(w) > 1]
    
    return vTokens

In [37]:
def correct_spelling(valTokens):
    
    # Discard null tweet
    if(len(valTokens) == 0):
        return ""
    
    # Long word check, discard tweet if there is a long word
    if(len([w for w in valTokens if len(w) > 40]) > 0):
        return ""
    
    # Misspelled words correction
    vMisspelled = _spell.unknown(valTokens)
    if(len(vMisspelled) > 0):
        valTokens = [w if not w in vMisspelled else _spell.correction(w) for w in valTokens]
        # Stopword drop
        
    valTokens = [w for w in valTokens if not w in _stop_words]
        
    return valTokens

In [38]:
def lemmatize(valTokens):
    return [_lemmatizer.lemmatize(w) for w in valTokens] 

In [39]:
def remove_repeated(valTokens):
    return ' '.join(sorted(set(valTokens), key=valTokens.index))

In [40]:
_texts = []
_sentiments = []
_ids = []
aux = _from_tweet + _packet_size
_to_tweet = aux if aux < len(_df['text']) else len(_df['text'])
for i in range(_from_tweet,_to_tweet):
    print(_df['text'][i])
    vTweet = preprocess_tweet(_df['text'][i])
    vTweet = correct_spelling(vTweet)
    vTweet = lemmatize(vTweet)
    vTweet = remove_repeated(vTweet)
    if vTweet != "":
        _texts.append(vTweet)
        _sentiments.append(_df['sentiment'][i])
        _ids.append(_df['id'][i])

the hills  last one D:
teamviewer is entertaining 
i intensely miss my bestfriends! much love!!  [two more days]
Watching the Hills season finale!!! Can't wait for the world premier of the New Moon clip!!!! 
The Hills and MTV music awards  what a way to end my weekend!
@KempEquine thanks  It was a lot of fun!
@JerZDre nothin much doin' the same--&gt;its a Sunday I feel like I'm breaking a rule by doin' anything today besides chill 
@KempEquine thanks  It was a lot of fun!
is lovin the sun 
just one hour left to the awards  ... watchin' the red carpet right now! @ashleytisdale u look amazing 


In [41]:
_cleaner_df = pd.DataFrame(_texts,columns=['text'])
_cleaner_df['sentiment'] = _sentiments
_cleaner_df['id'] = _ids
_cleaner_df.to_csv('cooked_tweets_' + str(_from_tweet) + '_' + str(_to_tweet) + '.csv', index = None, encoding='utf-8')

(10, 3)